In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import folium
from folium.plugins import HeatMap
from scipy.stats import norm

import matplotlib.colors as mc # For the legend
from matplotlib.cm import ScalarMappable

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_excel("Tech Vend data FY23 Anon.xlsx", engine="openpyxl")
df.head()

In [ ]:
df = df[~df.duplicated(keep = False)]
# removes duplicates

In [ ]:
df.sort_values(by=["Machine","Date"], inplace=True)
df.reset_index(drop=True,inplace=True)
# Algorithm to assign user IDs
current_user_id = 0
df['userID'] = None

for index, row in df.iterrows():
    if index == 0 or \
            (row['Date'] - df.iloc[index - 1]['Date']).seconds // 60 >= 5 or \
            any(row[['Cost Center Level 5', 'Cost Center Level 4', 'Management Level', 'Machine']] != df.iloc[index - 1][['Cost Center Level 5', 'Cost Center Level 4', 'Management Level', 'Machine']]):
        current_user_id += 1
    df.at[index, 'userID'] = current_user_id
print(df.head(100))

In [ ]:
#get the median cost of each item, and replace all costs for that item with the median cost.
#make sure to be careful with items that have 10 or less entries
counts = df.groupby("Item Desc").count()
for item in counts.index:
    if counts.loc[item]["Date"] > 10:
        df.loc[df["Item Desc"] == item,"Cost"] = df[df["Item Desc"] == item]["Cost"].median()
    else:
        #no other changes are needed
        print("Item (%s) Info:"%(item))
        print(df[df["Item Desc"] == item]["Cost"])

In [ ]:
machine_coordinates = {
    'London': {'Latitude': 51.5072, 'Longitude': -0.1275},
    'Belfast': {'Latitude': 54.5964, 'Longitude': -5.93},
    'Reading': {'Latitude': 51.45625, 'Longitude': -0.97113},
    'Birmingham': {'Latitude': 52.48, 'Longitude': -1.9025},
    'Leeds': {'Latitude': 53.801277, 'Longitude': -1.548567},
    'Bristol': {'Latitude': 51.4536, 'Longitude': -2.5975},
    'Manchester': {'Latitude': 53.4794, 'Longitude': -2.2453},
    'Watford': {'Latitude': 51.655, 'Longitude': -0.3957},
    'Edinburgh': {'Latitude': 55.953251, 'Longitude': -3.188267},
    'Glasgow': {'Latitude': 55.860916, 'Longitude': -4.251433},
    'Cardiff': {'Latitude': 51.4837, 'Longitude': 3.1681}
}

df['Latitude'] = df['Machine'].apply(lambda x: machine_coordinates[x.split()[0]]['Latitude'])
df['Longitude'] = df['Machine'].apply(lambda x: machine_coordinates[x.split()[0]]['Longitude'])

print(df.head(100000))

In [ ]:
machine_coordinates = {
    'London Office 1': {'Latitude': 51.5081, 'Longitude': -0.1248},
    'London Office 2': {'Latitude': 51.5045, 'Longitude': 0.0865},
    'Belfast': {'Latitude': 54.5964, 'Longitude': -5.93},
    'Reading': {'Latitude': 51.45625, 'Longitude': -0.97113},
    'Birmingham': {'Latitude': 52.48, 'Longitude': -1.9025},
    'Leeds': {'Latitude': 53.801277, 'Longitude': -1.548567},
    'Bristol': {'Latitude': 51.4536, 'Longitude': -2.5975},
    'Manchester': {'Latitude': 53.4794, 'Longitude': -2.2453},
    'Watford': {'Latitude': 51.655, 'Longitude': -0.3957},
    'Edinburgh': {'Latitude': 55.953251, 'Longitude': -3.188267},
    'Glasgow': {'Latitude': 55.860916, 'Longitude': -4.251433},
    'Cardiff': {'Latitude': 51.4816, 'Longitude': 3.1791}
}

df['Latitude'] = df['Machine'].apply(lambda x: machine_coordinates[' '.join(x.split()[:3])]['Latitude'] if x.startswith('London') else machine_coordinates[x.split()[0]]['Latitude'])
df['Longitude'] = df['Machine'].apply(lambda x: machine_coordinates[' '.join(x.split()[:3])]['Longitude'] if x.startswith('London') else machine_coordinates[x.split()[0]]['Longitude'])

print(df.head(100)) 

In [ ]:
df.loc[df['Machine'].str.startswith('Cardiff'), 'Latitude'] = 51.4816
df.loc[df['Machine'].str.startswith('Cardiff'), 'Longitude'] = -3.1791


print(df[df['Machine'].str.startswith('Cardiff')])

In [ ]:
condition_office_1 = df['Machine'].str.startswith('London Office 1')
condition_office_2 = df['Machine'].str.startswith('London Office 2')
df.loc[condition_office_1, 'City'] = 'London Office 1'
df.loc[condition_office_2, 'City'] = 'London Office 2'

df.loc[~(condition_office_1 | condition_office_2), 'City'] = df['Machine'].str.split().str[0]
print(df)

In [ ]:
#Map Heatmap for sum of cost per city
location_costs = df.groupby(['Latitude', 'Longitude'])['Cost'].sum().reset_index()

heatmap_map = folium.Map(location=[54.7023545, -3.2765753], zoom_start=6)

heat_data = [[row['Latitude'], row['Longitude'], row['Cost']] for index, row in location_costs.iterrows()]

HeatMap(heat_data).add_to(heatmap_map)

heatmap_map.save('total_cost_heatmap.html')

In [ ]:
#bar chart for sum of cost per city
city_costs = df.groupby('City')['Cost'].sum().reset_index()

city_costs_sorted = city_costs.sort_values(by='Cost', ascending=False)

fig = px.bar(city_costs_sorted, x='City', y='Cost', title='Total Cost by City', labels={'City': 'City', 'Cost': 'Total Cost'})

fig.show()

In [ ]:
#Heatmap for relation between UserID and cost
fig = px.density_heatmap(df, x='userID', y='Cost', nbinsx=200, nbinsy=200)
fig.update_layout(title='HeatMap to show userID Relation to Cost')
fig.show()

In [ ]:
#Scatter for Cost vs. UserID (Colored by Management Level)
filtered_df = df[~df['Management Level'].isin(['Specialist', 'Intern/Trainee', 'Administrative'])]

fig = px.scatter(filtered_df, x='userID', y='Cost', color='Management Level',
                 labels={'userID': 'User ID', 'Cost': 'Cost', 'Management Level': 'Management Level'},
                 title='Cost vs. UserID (Colored by Management Level)')

fig.show()

In [ ]:
#Scatter for Cost vs. UserID (Colored by City)'
fig = px.scatter(df, x='userID', y='Cost', color='City',
                 labels={'userID': 'User ID', 'Cost': 'Cost', 'City': 'City'},
                 title='Cost vs. UserID (Colored by City)')

fig.show()

In [ ]:
unique_costs = df['Cost'].unique()
print(unique_costs)

In [ ]:
#stacked bar chart for total cost by mangement level within each cost center
filtered_df = df[~df['Management Level'].isin(['Specialist', 'Intern/Trainee', 'Administrative'])]
grouped_data = filtered_df.groupby(['Cost Center Level 5', 'Management Level'], as_index=False)['Cost'].sum()

fig = px.bar(grouped_data, x='Cost Center Level 5', y='Cost', color='Management Level',
             title='Total Cost by Management Level within each Cost Center Level 5',
             labels={'Cost Center Level 5': 'Cost Center Level 5', 'Cost': 'Total Cost', 'Management Level': 'Management Level'},
             barmode='stack')
fig.show()

In [ ]:
#stacked bar chart for total cost by city within each cost center
grouped_data = filtered_df.groupby(['Cost Center Level 5', 'City'], as_index=False)['Cost'].sum()
fig = px.bar(grouped_data, x='Cost Center Level 5', y='Cost', color='City',
             title='Total Cost by City within each Cost Center Level 5',
             labels={'Cost Center Level 5': 'Cost Center Level 5', 'Cost': 'Total Cost', 'City': 'City'},
             barmode='stack')

fig.show()

In [ ]:
#Stacked bar chart for total cost by cost center within each city
filtered_df = df[~df['Management Level'].isin(['Specialist', 'Intern/Trainee', 'Administrative'])]

grouped_data = filtered_df.groupby(['City', 'Cost Center Level 5'], as_index=False)['Cost'].sum()

fig = px.bar(grouped_data, x='City', y='Cost', color='Cost Center Level 5',
             title='Total Cost by Cost Center Level 5 within each City',
             labels={'City': 'City', 'Cost': 'Total Cost', 'Cost Center Level 5': 'Cost Center Level 5'},
             barmode='stack')

fig.show()

In [ ]:
#Map Heatmap for average cost per city
location_avg_cost = df.groupby(['Latitude', 'Longitude'])['Cost'].mean().reset_index()

heatmap_map = folium.Map(location=[54.7023545, -3.2765753], zoom_start=6)

heat_data = [[row['Latitude'], row['Longitude'], row['Cost']] for index, row in location_avg_cost.iterrows()]

HeatMap(heat_data).add_to(heatmap_map)

heatmap_map.save('avg_cost_heatmap.html')

In [ ]:
#Total cost Bar chart per item
item_counts = df['Item Desc'].value_counts().reset_index()
item_counts.columns = ['Item Desc', 'Frequency']
fig = px.bar(item_counts, x='Item Desc', y='Frequency', title='Histogram of Item Descriptions by Popularity')
fig.update_layout(xaxis={'type': 'category', 'tickangle': 270})
fig.show()

In [ ]:
#Total cost bar chart for top 8 items
item_stats = df.groupby('Item Desc').agg(Frequency=('Item Desc', 'count'), Average_Cost=('Cost', 'mean')).reset_index()
top_8_items = item_stats.nlargest(8, 'Frequency')

fig = px.bar(top_6_items, x='Item Desc', y='Frequency', title='Top 8 Items by Popularity (with respective cost of each as label)',
             text='Average_Cost', hover_data=['Average_Cost'])

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    xaxis={'type': 'category', 'tickangle': 270},
    yaxis_title="Frequency",
    hovermode='closest',
    height=800
)

fig.show()

In [ ]:
#Tree Chart for popularity of item category
fig = px.treemap(item_counts, 
                 path=['Category'], 
                 values='Frequency', 
                 title='Treemap of Item Category by Popularity',
                 color='Frequency', 
                 color_continuous_scale=px.colors.sequential.Blues,  
                 custom_data=['Category', 'Frequency'])  

fig.update_traces(texttemplate='<b>%{label}</b><br>%{customdata[1]}', 
                  textposition='middle center') 


fig.update_layout(
    paper_bgcolor='rgba(255,255,255,1)',  
    plot_bgcolor='rgba(255,255,255,1)', 
    uniformtext=dict(minsize=10)         
)

fig.show()

In [ ]:
#Frequency of Mangement Levels bar chart
management_counts = df['Management Level'].value_counts().reset_index()
management_counts.columns = ['Management Level', 'Frequency']
​
fig = px.bar(management_counts, x='Management Level', y='Frequency', 
             title='Frequency of Management Levels', 
             labels={'Management Level': 'Management Level', 'Frequency': 'Frequency'})
fig.update_layout(xaxis={'type': 'category', 'categoryorder': 'total descending'})
fig.show()

In [ ]:
#Number of Unique users by management level bar chart
management_counts = df.groupby('Management Level')['userID'].nunique().reset_index()
management_counts.columns = ['Management Level', 'Unique UserIDs']

fig = px.bar(management_counts, x='Management Level', y='Unique UserIDs',
             title='Number of Unique UserIDs by Management Level',
             labels={'Management Level': 'Management Level', 'Unique UserIDs': 'Number of Unique UserIDs'})

fig.update_layout(xaxis={'type': 'category', 'categoryorder': 'total descending'})

fig.show()

In [ ]:
#Total cost by cost center bar chart
cost_center_totals = df.groupby('Cost Center Level 5', as_index=False)['Cost'].sum()
fig = px.bar(cost_center_totals,
             x='Cost Center Level 5',
             y='Cost',
             title='Total Cost by Cost Center Level 5',
             labels={'Cost': 'Total Cost', 'Cost Center Level 5': 'Cost Center Level 5'})
fig.update_layout(xaxis_title="Cost Center Level 5",
                  yaxis_title="Total Cost")
fig.show()

In [ ]:
#Total cost by managment level bar chart
cost_center_totals = df.groupby('Management Level', as_index=False)['Cost'].sum()

fig = px.bar(cost_center_totals,
             x='Management Level',
             y='Cost',
             title='Total Cost by Management Level',
             labels={'cost': 'Total Cost', 'Management Level': 'Management Level'})


fig.update_layout(xaxis_title="Management Level",
                  yaxis_title="Total Cost")
fig.show()